In [67]:
import dash
from dash import Dash, html, dcc, callback, Output, Input, State
from dash import dash_table
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd


In [68]:
athlete_events = pd.read_csv("./Data/athlete_events.csv")
athlete_events.head(2)

best_italian_years = pd.read_excel("./Data/best_italian_years.xlsx")
best_italian_years

,Year,Season,Number of medals
0,2004,Summer,104
1,1960,Summer,85
2,1920,Summer,84


In [69]:
medals = athlete_events[athlete_events['Medal'].isin(['Gold', 'Silver', 'Bronze'])]
medals_count = medals.groupby(['Sport', 'Team', 'Medal']).size().reset_index(name='Count')



italy_medals = medals.query("Team == 'Italy'")
italy_medals.value_counts("Medal")
italy_medals_count = italy_medals.value_counts("Sport")

medals_per_game = italy_medals.groupby("Year")[["Season"]].value_counts()
medals_per_game.sort_values(ascending=False, inplace=True)
medal_df = medals_per_game.head(3)  

age_count = italy_medals["Age"].value_counts()

italy_athletes = athlete_events.query("Team == 'Italy'")
all_italy_athletes = italy_athletes["Age"].value_counts()

italy_events = athlete_events.query("Team == 'Italy'")
sport_heights_mean = italy_events.groupby("Sport").agg({'Height': 'mean'})
sport_heights_mean = sport_heights_mean.dropna()
sport_heights_mean.sort_values(by="Height", inplace=True)

italy_events["BMI"] = italy_events["Weight"] / ((italy_events["Height"]/100)**2)









# For the ones I did yesterday
all_sports = medals['Sport'].unique()

C:\Users\Student\AppData\Local\Temp\ipykernel_26140\193752641.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
# Italy medals

fig_italy_medals = px.bar(
    italy_medals_count,
    title="Italy medals",
    labels={"value": "Number of medals", 
            "variable": "Medals", 
            "count": "Number of medals"},
    template="simple_white"
    )

# Total number of medals awarded athletes per age group in Italy

fig_age_medals = px.histogram(
    age_count,
    x=age_count.index,
    y=age_count.values,
    nbins=len(age_count.index),
    labels={
        "Age": "Age Group", 
        "sum of y": "Number of Athletes"
    },
    title="Total number of medals awarded athletes per age group in Italy",
    template="simple_white",
)

fig_age_medals.update_traces(hovertemplate=None)
fig_age_medals.update_layout(hovermode="x")
fig_age_medals.update_layout(yaxis_title="Number of medals")

# Total number of athletes per age group in Italy

fig_athletes_age_groups = px.histogram(
    all_italy_athletes,
    x=all_italy_athletes.index,
    y=all_italy_athletes.values,
    nbins=len(all_italy_athletes.index),
    title="Total number of athletes per age group in Italy",
    template="simple_white"
)

fig_athletes_age_groups.update_traces(hovertemplate=None)
fig_athletes_age_groups.update_layout(hovermode="x")
fig_athletes_age_groups.update_layout(yaxis_title="Number of athletes")

# Average Italian athletes' height per sport

fig_athlete_height_sport = px.bar(
    sport_heights_mean,
    x=sport_heights_mean.index,
    y="Height",
    title="Average athlete height per sport",
    template="simple_white",
)

fig_athlete_height_sport.update_xaxes(tickangle=-45)
fig_athlete_height_sport.update_yaxes(range=[150, 193])

In [71]:
app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.CYBORG],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Olympic Games Dashboard", className='text-center text-primary mx-3')
            ], width=12, style={'text-align': 'center'})
    ], className="mt-5"),
    
    # Age distribution headings
    dbc.Row([
        dbc.Col([
            html.H4("Gender distribution historically")
        ], width={"size": 5, "offset": 0}, style={'text-align': 'center'}),
        
        dbc.Col([
            html.H4("Gender distribution 2016")
        ], width={"size": 5, "offset": 0}, style={'text-align': 'center'}),
    ], justify='center'),
    
    # Age distribution plots
    dbc.Row([
        dbc.Col([
            html.Img(src=dash.get_asset_url('Age distribution overall.png')) 
        ], width={"size": 5, "offset": 0}, style={'text-align': 'center'}),
        dbc.Col([
            html.Img(src=dash.get_asset_url('Age distribution 2016.png')) 
        ], width={"size": 5, "offset": 0}, style={'text-align': 'center'})
    ], justify='center'),
    
    # Italy medals
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=fig_italy_medals)
        ], className="mt-5")
    ], justify='center'),
    
    # Italy medals by season
    dbc.Row([
        dbc.Col([
            html.Img(src=app.get_asset_url('Italy medals by year overview.png')),
        ], width={"size": 12, "offset": 0}, style={'text-align': 'center'}),
    ], justify='center', className="mt-5"),
    
    # Data table of the best Italian years
    dbc.Row([
        dbc.Col([
            dash_table.DataTable(
                id='best_italian_years',
                data=best_italian_years.to_dict('records'),
                columns=[{"name": i, "id": i} for i in best_italian_years.columns])
        ], width={"size": 10, "offset": 0}, className="text-black-50")
    ], justify='center', className="mt-5"),
    
    # Medals per age group in Italy
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=fig_age_medals)
        ], className="mt-5")
    ], justify='center'),
    
    # Athletes per age groups Italy
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=fig_athletes_age_groups)
        ], className="mt-5")
    ], justify='center'),
    
    # Athlete heights per sport Italy
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=fig_athlete_height_sport)
        ], className="mt-5")
    ], justify='center'),
])  




fig_age_medals
      
#     dbc.Row([
#         dbc.Col([
#             dcc.Dropdown(
#                 id='sport_selector',
#                 options=[{'label': sport, 'value': sport} for sport in all_sports],
#                 value=all_sports[0]
#             ),
#             dcc.Graph(id='medal_graph')
#         ]),
#         dbc.Col([
#             dcc.Dropdown(
#                 id='sport_selector2',
#                 options=[{'label': sport, 'value': sport} for sport in all_sports],
#                 value=all_sports[0]
#             ),
#             dcc.Graph(id='age_graph')
#         ])
#     ])
# ])

# @callback(
#     Output('medal_graph', 'figure'),
#     Input('sport_selector', 'value')
# )
# def update_medal_graph(selected_sport):
#     medals_df = medals_count[medals_count['Sport'] == selected_sport]
#     fig = px.bar(medals_df, 
#                  x="Team", y="Count", 
#                  color="Medal", 
#                  title=f"Medals for {selected_sport}", 
#                  template="simple_white", 
#                  labels={"Team": "Country", "Count": "Number of medals"})
#     fig.update_xaxes(tickangle=-45)
#     return fig

# @callback(
#     Output('age_graph', 'figure'),
#     Input('sport_selector2', 'value')
# )
# def update_age_graph(selected_sport):
#     sport_df = athlete_events[athlete_events['Sport'] == selected_sport]
#     age_distribution = sport_df.groupby('Age').size().reset_index(name='Count')
#     fig = px.bar(age_distribution, 
#                  x="Age", y="Count", 
#                  color="Age", 
#                  title=f"Age distribution for {selected_sport}", 
#                  template="simple_white",
#                  labels={"Count": "Number of athletes"})
#     return fig

app.run(debug=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
